<a href="https://colab.research.google.com/github/MichalMichniak/Sieci-Neuronowe/blob/main/agh_ai_days_rl_intro_tabular_approximation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tabular approximation with Tic-Tac-Toe

Now, let's deal with something slightly more advanced:
a game of multiple states and discrete actions: Tic-Tac-Toe.

Let's try to solve this problem!

<img src="https://mk.ssb-media.com/images/alt_368741_1_2x.jpg" alt="drawing" width="400px"/>

## Credits
- Interactive Tic-Tac-Toe example has been taken from [this Jupyter Notebook](https://colab.research.google.com/github/asolin/notebooks/blob/main/notebooks/RL-for-Tic-Tac-Toe.ipynb) by [Arno Solin](https://github.com/asolin/).
- Maciej Aleksandrowicz for AGH AI Days 2024

### Further resources
- Another source for Tabular Q-Learning for Tic-Tac-Toe can be find in [this Github repository](https://github.com/sunbri/tictactoe) by [Brian Sun](https://github.com/sunbri).

## Interactive Tic-Tac-Toe

First, let's import an example of interactive game into this notebook. Play with it for a while.

In [1]:
import numpy as np
import random

### Code for interactive game
 * Run cell and hide this section with the environement defition.
 * Note, that in this `O` and `X` notation, the `X`s always starts.

In [ ]:
'''
   Copyright 2017 Neil Slater
   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at
       http://www.apache.org/licenses/LICENSE-2.0
   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.

   The original code as been modified for educational purposes.
   Further modifications for education were made in 2024 for AGH AI Days.
'''

class TicTacToeGame():
    def __init__(self):
        self.state = "         "  #a string of length 9 that encode the state of the 3*3 board
        self.player = "X"
        self.winner = None

    def allowed_moves(self):      #find the empty position in the state string
        states = []               #store all possible next states
        for i in range(len(self.state)):
            if self.state[i] == ' ':
                states.append(self.state[:i] + self.player + self.state[i+1:])
        return states

    def make_move(self, next_state):
        if self.winner:
            raise(Exception("Game already completed, cannot make another move!"))
        if not self.__valid_move(next_state):
            raise(Exception("Cannot make move {} to {} for player {}".format(
                    self.state, next_state, self.player)))

        self.state = next_state
        self.winner = self.predict_winner(self.state)
        if self.winner:
            self.player = None
        elif self.player == 'X':
            self.player = 'O'
        else:
            self.player = 'X'

    def playable(self):
        return ( (not self.winner) and any(self.allowed_moves()) )

    def predict_winner(self, state):
        lines = [(0,1,2), (3,4,5), (6,7,8), (0,3,6), (1,4,7), (2,5,8), (0,4,8), (2,4,6)]  # all possible lines
        winner = None
        for line in lines:
            line_state = state[line[0]] + state[line[1]] + state[line[2]]
            if line_state == 'XXX':
                winner = 'X'
            elif line_state == 'OOO':
                winner = 'O'
        return winner

    def __valid_move(self, next_state):
        allowed_moves = self.allowed_moves()  #get all possible next states
        if any(state == next_state for state in allowed_moves): #check if the input next_state is in
            return True
        return False

    def print_help(self):
        s = tuple([x + 1 for x in range(9)])
        self._print_state(s)

    def print_board(self):
        s = self.state
        self._print_state(s)

    def _print_state(self, s):
        print(f"  {s[0]} | {s[1]} | {s[2]} ")
        print(f" -----------")
        print(f"  {s[3]} | {s[4]} | {s[5]} ")
        print(f" -----------")
        print(f"  {s[6]} | {s[7]} | {s[8]} ")


### Code for an agent
 * Inspect code for all TODOs to resolve.
 * Run cell and hide this section with the agent defition - we will take about it more later.


In [ ]:
class Agent():
    def __init__(self, game_class, epsilon=0.1, alpha=0.5, value_player='X'):\
        # -------------------------------------------------------------------- #
        # TODO: initalize an empty dict (our table for values)
        self.V = None
        self.V = dict()
        # -------------------------------------------------------------------- #
        self.NewGame = game_class
        self.epsilon = epsilon
        self.alpha = alpha
        self.value_player = value_player

    def state_value(self, game_state: str) -> float:
        # -------------------------------------------------------------------- #
        # TODO: get value of a state, or return 0.0
        # self.V.get(...)
        # return 0.0

        # -------------------------------------------------------------------- #

    def learn_game(self, num_episodes=1000):
        for episode in range(num_episodes):
            self.learn_from_episode()

    def learn_from_episode(self):
        game = self.NewGame()
        _, move = self.learn_select_move(game)
        while move:
            move = self.learn_from_move(game, move)

    def learn_from_move(self, game, move):
        game.make_move(move)
        r = self.__reward(game)
        td_target = r
        next_state_value = 0.0
        selected_next_move = None
        if game.playable():
            best_next_move, selected_next_move = self.learn_select_move(game)
            next_state_value = self.state_value(best_next_move)
        current_state_value = self.state_value(move)
        td_target = r + next_state_value

        # -------------------------------------------------------------------- #
        # TODO: Implement update equation: V(a) = V(a) + alpha * (TARGET - V(a))
        # where:
        # a => move
        # V => self.V
        # alpha => self.alpha
        # target => td_target
        # reading V value => current_state_value
        # ----
        # self.V[...] = ...

        # -------------------------------------------------------------------- #
        return selected_next_move

    def learn_select_move(self, game):
        allowed_state_values = self.__state_values( game.allowed_moves() )
        if game.player == self.value_player:
            best_move = self.__argmax_V(allowed_state_values)
        else:
            best_move = self.__argmin_V(allowed_state_values)

        selected_move = best_move
        if random.random() < self.epsilon:  #epsilon-greedy strategy
            selected_move = self.__random_V(allowed_state_values)

        return (best_move, selected_move)

    def play_select_move(self, game):
        allowed_state_values = self.__state_values( game.allowed_moves() )
        if game.player == self.value_player:
            return self.__argmax_V(allowed_state_values)
        else:
            #return self.__argmin_V(allowed_state_values)
            return self.__random_V(allowed_state_values)

    def demo_game(self, verbose=False):
        game = self.NewGame()
        t = 0
        while game.playable():
            if verbose:
                print(f" \nTurn {t}\n")
                game.print_board()
            move = self.play_select_move(game)
            game.make_move(move)
            t += 1
        if verbose:
            print(" \nTurn {}\n".format(t))
            game.print_board()
        if game.winner:
            if verbose:
                print("\n{} is the winner!".format(game.winner))
            return game.winner
        else:
            if verbose:
                print("\nIt's a draw!")
            return '-'

    def interactive_game(self, agent_player='X'):
        game = self.NewGame()
        t = 0
        while game.playable():
          if t == 0:
            print(" \nBOARD PLACES\n")
            game.print_help()
          print(" \nTurn {}\n".format(t))
          game.print_board()
          if game.player == agent_player:
              move = self.play_select_move(game)
              game.make_move(move)
          else:
              move = self.__request_human_move(game)
              game.make_move(move)
          t += 1

        print(" \nTurn {}\n".format(t))
        game.print_board()

        if game.winner:
            print("\n{} is the winner!".format(game.winner))
            return game.winner
        print("\nIt's a draw!")
        return '-'

    def round_V(self):
        # After training, this makes action selection random from equally-good choices
        for k in self.V.keys():
            self.V[k] = round(self.V[k],1)


    def __state_values(self, game_states):
        return dict((state, self.state_value(state)) for state in game_states)

    def __argmax_V(self, state_values):
        max_V = max(state_values.values())
        chosen_state = random.choice([state for state, v in state_values.items() if v == max_V])
        return chosen_state

    def __argmin_V(self, state_values):
        min_V = min(state_values.values())
        chosen_state = random.choice([state for state, v in state_values.items() if v == min_V])
        return chosen_state

    def __random_V(self, state_values):
        return random.choice(list(state_values.keys()))

    def __reward(self, game):
        # -------------------------------------------------------------------- #
        # TODO: implement SPARSE reward for winning and SPARSE penalty for losing
        reward = None
        if game.winner == self.value_player:
            reward = None
        elif game.winner:
            reward = None
        else:
            reward = None
        return reward
        # -------------------------------------------------------------------- #

    def __request_human_move(self, game):
        allowed_moves = [i+1 for i in range(9) if game.state[i] == ' ']
        human_move = None
        while not human_move:
            idx = int(input('Choose move for {}, from {} : '.format(game.player, allowed_moves)))
            if any([i==idx for i in allowed_moves]):
                human_move = game.state[:idx-1] + game.player + game.state[idx:]
        return human_move

def demo_game_stats(agent, games:int=10000):
    results = [agent.demo_game() for i in range(games)]
    game_stats = {k: results.count(k)/100 for k in ['X', 'O', '-']}
    print("    percentage results: {}".format(game_stats))

## Make an Agent
Note, that in the environemnt defition (above) X always starts.

Let's create an Agent, setup some learning parameters, select `O`/`X` symbol for it.

Then, evaluate the "empty" policy of the agent by playing $10000$ iterations of game. The print out of the efficiency of this AI model will present wins, loses and draws (noted as `-`).

In [ ]:
# Create you agent (named 'X') and assign the learning rate and discount factor
agent = Agent(TicTacToeGame, epsilon = 0.1, alpha = 0.5, value_player='X')

print("Before learning:")
demo_game_stats(agent)

Try it face-2-face in the interactive mode:

In [ ]:
agent.interactive_game()

## Train the agent

You can have the agent play against itself for a number of games to learn how to play the game. In the cell below, you will make the agent play 1000 games. After that, try playing against it again. Can you see any difference?

In [ ]:
agent.learn_game(1000)
print("After 1000 learning games:")
demo_game_stats(agent)

In [ ]:
agent.interactive_game()

## Task 1. More training and playing
Continue training the method:
* First for 4000 more games, then try playing again.
* Finally, keep training the model for 10000 games more and try playing again.

What do you notice? Is the agent getting harder to beat?

In [ ]:
# ------------------- #
# TODO: code goes here

# ------------------- #

In [ ]:
agent.interactive_game()

## Task 2. Tweak the model
Now you can tweak the model, and experiment further:
1. Start by changing the AI identity to "O" (so that you get to start the games). Re-train and re-try playing.
2. Experiment with changing the learning rate and discounting factors.
3. Try to plot the improvement proces in function of episodes.

In [ ]:
# ------------------- #
# TODO: experiments goes here

# ------------------- #